In [3]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import seaborn as sns
import sklearn as sk
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
from sklearn.metrics import log_loss
from sklearn.metrics import brier_score_loss
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [4]:
games = pd.read_csv('games_new_vars.csv')

Here, we will fit a few logistic regression models to predict the probability of a higher rating player winning. 
Order of business: 

1) Perform k-fold cross validation (k = 5) for a simple logistic regression. Predictor will be absolute value of difference (abs_diff_rating) 
in rating, and response will be whether the higher rating won (higher_rating_won). 

2) Get familiar with the multiple logistic regression code before mass producing in a python file. 

In [5]:
# Empty model
lg = LogisticRegression()

In [15]:
# Perform k-fold cross validation for k = 5
kf_five = KFold(n_splits = 5)

# We will find the estimated_mse
est_err_five = 0
for train, test in kf_five.split(games):
    
    # Create training and test data
    train_data = games.iloc[train][['abs_diff_rating', 'higher_rating_won']]
    test_data = games.iloc[test][['abs_diff_rating', 'higher_rating_won']]
    
    # Predictor and response (training)
    X_train = DataFrame(train_data.abs_diff_rating)
    Y_train = DataFrame(train_data.higher_rating_won)
    
    # Fit the model
    lg.fit(X_train, Y_train)
    
    # Predictor and response(test)
    X_test = DataFrame(test_data.abs_diff_rating)
    Y_test = DataFrame(test_data.higher_rating_won)
    
    # Store the score for the model
    est_err_five += lg.score(X_test, Y_test)
    
# Take the average MSE
est_err_five = est_mse_five / 5

# TODO Ugly dataConversionWarning, fixed in python file
# TODO Not MSE here but the ERR being estimated, fixed in python file

/Users/Leonard/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Leonard/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Leonard/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Leonard/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

In [16]:
est_err_five

0.62483754172536

This metric tells me that the model will properly predict a win or los 62% of the time. 

But I want more! I'm going to try and add two more variables into our logistic regression here. 

1) White is known to have an advantage (how subtle, you can look up here: TODO) over black by getting to move first. So, I intend to code a white_higher_rated
(white higher rating) binary variable. 1 if White had the higher rating 0 otherwise. This may help show how being white or black affects having a higher rating over the other player. 

2) Number of turns. I hypothesize that longer turned games usually result in more venly matched games, despite the rating. Perhaps by knowing the number of turns made, I can say something about the probability of the higher rated player winning. 

-> Come in Bayes? Information. Random thought about conditional probabilities here. 

In [23]:
# Create the white_higher_rated variable
# If white had the higher rating, we get a 1, otherwise 1. 
games['white_higher_rated'] = 0
games.loc[games.higher_rating == 'white', 'white_higher_rated'] = 1

In [8]:
# Fit a multiple logistic regression model 
lg_mult = LogisticRegression()

In [24]:
# Trying out multiple predictors. 
X = DataFrame(games[['abs_diff_rating', 'higher_rating_coded', 'turns', 'white_higher_rated']])
Y = DataFrame(games['higher_rating_won'])

In [25]:
lg_mult.fit(X, Y)

/Users/Leonard/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
lg_mult.score(X, Y)

0.6358061621298235

So, about a percentage point increase in performance. Nice. Not bad. 